### Замерим качество Линейной регрессии после обработки данных не просто на отложенной выборке, но и на Кросс-Валидации на 4 фолдах!

In [94]:
import numpy as np
import pandas as pd

processed_data = pd.read_csv('processed_data.csv', index_col='id')

In [95]:
processed_data.head()


,vendor_id,passenger_count,store_and_fwd_flag,distance_km,log_trip_duration
id,,,,,
id2875421,1,930.399753,0,1.500479,6.122493
id2377394,0,930.399753,0,1.807119,6.498282
id3858529,1,930.399753,0,6.392080,7.661527
id3504673,1,930.399753,0,1.487155,6.063785
id2181028,1,930.399753,0,1.189925,6.077642


#### ! Не перемешивайте данные

In [32]:
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_log_error
import numpy as np

selector = KFold(n_splits=4)

losses = []

for train_index, test_index in selector.split(processed_data):
    X_train, X_test = processed_data.iloc[train_index][['vendor_id', 'passenger_count', 'store_and_fwd_flag', 'distance_km']], processed_data.iloc[test_index][['vendor_id', 'passenger_count', 'store_and_fwd_flag', 'distance_km']]
    y_train, y_test = processed_data.iloc[train_index]['log_trip_duration'], processed_data.iloc[test_index]['log_trip_duration']
    
    model = LinearRegression()
    model.fit(X_train, y_train)
    
    predictions = model.predict(X_test)
    squared_log_errors = (predictions - y_test) ** 2
    msle = np.mean(squared_log_errors)
    losses.append(msle)

mean_cv_loss = np.mean(losses)

print(f"{round(mean_cv_loss, 3)}")


0.426


## Поработал один из хитрых гномов!

В отличие от своих собратьев, третий гном оказался тем еще бездельником в школьные годы, но все равно страстно желал во всем догнать первых двух. И сейчас, желая помочь им в построении модели по предсказанию длительности поездки такси, добавил в данные 20 зашифрованных фичей (их смысл нам не рассказали: какая-то секретная информация о водителях).

Гном думал следующим образом: "Ну не может же модель стать хуже! А тут вот авось и мое нововведение уменьшит ошибку в разы! Тогда и меня станут звать на гномий  data-саммит."

Проверим на кросс-валидации, насколько гном оказался прав!

In [96]:
new_data = pd.read_csv('new_data.csv', index_col='id')

In [34]:
new_data.head()
columns = new_data.columns
print(columns)

Index(['vendor_id', 'passenger_count', 'store_and_fwd_flag', 'distance_km',
       'log_trip_duration', 'feature_1', 'feature_2', 'feature_3', 'feature_4',
       'feature_5', 'feature_6', 'feature_7', 'feature_8', 'feature_9',
       'feature_10', 'feature_11', 'feature_12', 'feature_13', 'feature_14',
       'feature_15', 'feature_16', 'feature_17', 'feature_18', 'feature_19',
       'feature_20'],
      dtype='object')


In [35]:
### Шаг №6
### Замерьте качество (MSLE, как и раньше) на Кросс-валидации, 
### используя MSE от log_trip_duration и назначенный ранее selector

### Your code is here
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_log_error
losses = []
splitter = KFold(n_splits=4)

for train_index, test_index in splitter.split(new_data):
    x_train, x_test = new_data.iloc[train_index][['feature_1', 'feature_2', 'feature_3', 'feature_4',
       'feature_5', 'feature_6', 'feature_7', 'feature_8', 'feature_9',
       'feature_10', 'feature_11', 'feature_12', 'feature_13', 'feature_14',
       'feature_15', 'feature_16', 'feature_17', 'feature_18', 'feature_19',
       'feature_20']], new_data.iloc[test_index][['feature_1', 'feature_2', 'feature_3', 'feature_4',
       'feature_5', 'feature_6', 'feature_7', 'feature_8', 'feature_9',
       'feature_10', 'feature_11', 'feature_12', 'feature_13', 'feature_14',
       'feature_15', 'feature_16', 'feature_17', 'feature_18', 'feature_19',
       'feature_20']]
    y_train,y_test = new_data.iloc[train_index]['log_trip_duration'],new_data.iloc[test_index]['log_trip_duration']

    model = LinearRegression()
    model.fit(x_train,y_train)

    predictions = model.predict(x_test)
    squared_log_errors = (predictions - y_test) ** 2
    msle = np.mean(squared_log_errors)
    losses.append(msle)

cross_val_error_2 = np.mean(losses)


print(f"MSLE на Кросс-валидации: {round(cross_val_error_2, 3)}")

MSLE на Кросс-валидации: 140.923


В линейной алгербре зачастую используют понятие **ранга матрицы**. Оно соответствует кол-ву линейно независимых столбцов в матрице. Иными словами, позволяет оценить, есть ли избыток информации в нашем датафрейме. Если ранг матрицы меньше, чем кол-во используемых столбцов, то некоторые фичи следует удалить, ведь иначе возникает ситуация строгой мультиколлинеарности.

Чтобы замерить ранг в наших матрицах объект-признак, можно воспользоваться функцией numpy.linalg.matrix_rank

Константным признаком в данном упражнении можно пренебречь.

In [71]:
processed_data = processed_data.drop(['log_trip_duration'], axis=1)
new_data = new_data.drop(['log_trip_duration'], axis=1)


In [72]:
columns = processed_data.columns
print(columns)

Index(['vendor_id', 'passenger_count', 'store_and_fwd_flag', 'distance_km'], dtype='object')


In [73]:
### Создайте переменные rank_processed, rank_new 
### Соответственно равные рангу изначальной матрицы
### с данными и рангу матрицы третьего гнома

### Your code is here
rank_processed = np.linalg.matrix_rank(processed_data)
rank_new = np.linalg.matrix_rank(new_data)


In [74]:
### Создайте переменные num_features_processed, num_features_new
### Соответственно равные кол-ву фичей в изначальной матрицы
### с данными и кол-ву фичей у третьего гнома

### Your code is here
num_features_processed = processed_data.shape[1]
num_features_new = new_data.shape[1]



In [75]:
### Шаг №7
print(f"В первой модели всего фичей: {num_features_processed}, - а ранг равен {rank_processed}")

print(f"Во второй модели всего фичей: {num_features_new}, - а ранг равен {rank_new}")

В первой модели всего фичей: 4, - а ранг равен 4
Во второй модели всего фичей: 24, - а ранг равен 5


Не кажется ли нам, что из-за новых 20 фичей появилась проблема мультиколлинеарности? Как поступить гному, чтобы, с одной стороны, получить адекватное качество, а с другой стороны, не повредить свое самолюбие и не убирать новые признаки?

Верно! Например, с помощью регуляризации.

Найдите такой параметр регуляризации $\alpha$ для Ridge и Lasso случая, чтобы ошибка MSLE на кросс-валидации оказалась строго меньше 0.4

**ALARM**: используйте процедуру масштабирования данных (воспользуйтесь методом MinMaxScaler) перед тем как применить регуляризацию. Важно - чтобы сохранить концепцию независимости обучения на трейне и на тесте, на каждой итерации кросс-валидации необходимо замерять параметры стандартизации исключительно на трейне, а потом применять на валидационном фолде.

In [113]:
### Пример, как это можно сделать в цикле
### То есть обучить Lasso, учитывая масштабирование
### Исключительно на трейне на каждой итерации
X = new_data.drop('log_trip_duration', axis=1)
Y = new_data['log_trip_duration']

from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import Lasso

scores = []

for train_index, test_index in selector.split(X):
    
    X_train, X_test = X.values[train_index], X.values[test_index]
    Y_train, Y_test = Y.values[train_index], Y.values[test_index]
    
    ### Фитим исключительно на трейне!
    scaler = MinMaxScaler()
    scaler.fit(X_train)
    
    ### Применяем обученный scaler на трейн и тест
    X_train_scaled = scaler.transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    ### max_iter иногда требуется ставить побольше, 
    ### особенно когда данных много и/или они сложные
    ### этот параметр регулирует верхнюю границу кол-ва
    ### итераций во время обучения
    ### подробнее - в документации
    
    ### По дефолту здесь параметр регуляризации alpha=1
    
    model_lasso = Lasso(max_iter=100000) 
    model_lasso.fit(X_train_scaled, Y_train)
    
    predictions = model_lasso.predict(X_test_scaled)
    
    scores.append(np.mean((predictions - Y_test)**2))

    
print(f"MSLE на Кросс-валидации равен: {np.mean(scores)}")

### P.S. если вы уже умеете работать с конструкциями
### Pipeline, GridSearchCV, cross_validate
### Можете использовать их. Мы познакомимся с ними позже.

MSLE на Кросс-валидации равен: 0.6332330617999488


In [112]:
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error

X = new_data.drop('log_trip_duration', axis=1)
Y = new_data['log_trip_duration']

from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import Lasso

scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

alphas = np.linspace(1e-5, 1e-4, 10)

best_alpha = None
best_score = float('inf')

for alpha in alphas:
    scores = []
    for train_index, test_index in selector.split(X):
        X_train, X_test = X_scaled[train_index], X_scaled[test_index]
        Y_train, Y_test = Y.values[train_index], Y.values[test_index]

        model_lasso = Lasso(alpha=alpha, max_iter=100000)
        model_lasso.fit(X_train, Y_train)

        predictions = model_lasso.predict(X_test)

        scores.append(mean_squared_error(Y_test, predictions))

    mean_score = np.mean(scores)
    if mean_score < 0.4 and mean_score < best_score:
        best_score = mean_score
        best_alpha = alpha

print(f"Лучший параметр регуляризации для Lasso: {best_alpha:.5f}")


Лучший параметр регуляризации для Lasso: 0.00004
